# импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import nltk
import re

from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\маша\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# загрузка файлов в датафрейм

In [2]:
df_orig = pd.read_json("original_news.json")
df_gen  = pd.read_json("generated_news.json")

df_orig = df_orig.rename(columns={"original_news": "text"})
df_gen  = df_gen.rename(columns={"generated_news": "text"})
df_orig["label"] = 0
df_gen["label"] = 1

df = pd.concat([df_orig, df_gen], ignore_index=True)
df = df[['text', 'label']]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.head()

,text,label
0,"В Грузии был задержан гражданин России, подозр...",1
1,Закон об исключительности украинского языка мо...,0
2,Российской валюте не стоит ожидать укрепления ...,0
3,4 сентября в Москве состоялось торжественное о...,1
4,В 2022 году в России зафиксированы значительны...,1


In [3]:
X = df['text']
y = df['label']

# функция извлечения стилометрических признаков

In [4]:
def extract_stylo_features(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    avg_sent_len = np.mean([len(word_tokenize(s)) for s in sentences]) if sentences else 0
    sent_len_var = np.var([len(word_tokenize(s)) for s in sentences]) if sentences else 0
    ttr = len(set(words)) / len(words) if words else 0
    markdown_bold = text.count("**")

    return pd.Series({
        "avg_sent_len": avg_sent_len,
        "sent_len_var": sent_len_var,
        "ttr": ttr,
        "markdown_bold": markdown_bold
    })

df_stylo = X.apply(extract_stylo_features)
df_features = pd.concat([df[['text']], df_stylo], axis=1)


# создание модели


In [5]:
tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    sublinear_tf=True
)

ct = ColumnTransformer([
    ("tfidf", tfidf, "text"),
    ("stylo", StandardScaler(), ["avg_sent_len", "sent_len_var", "ttr", "markdown_bold"]),
])

model = Pipeline([
    ('features', ct),
    ('clf', LogisticRegression(max_iter=400, class_weight="balanced"))
])


# обучение и оценка качества

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df_features, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9270833333333334
F1-score: 0.9195402298850575
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       106
           1       0.91      0.93      0.92        86

    accuracy                           0.93       192
   macro avg       0.93      0.93      0.93       192
weighted avg       0.93      0.93      0.93       192



In [7]:
feature_names = model.named_steps['features'].named_transformers_['tfidf'].get_feature_names_out()
coefs = model.named_steps['clf'].coef_[0][:len(feature_names)]

top_ai = np.argsort(coefs)[-20:][::-1]  
top_human = np.argsort(coefs)[:20]      

print("\n🔹 Маркеры AI:")
for i in top_ai:
    print(feature_names[i], round(coefs[i], 4))

print("\n🔹 Маркеры Human:")
for i in top_human:
    print(feature_names[i], round(coefs[i], 4))



🔹 Маркеры AI:
инцидент 0.6905
однако 0.6749
этот 0.6451
это 0.6394
для 0.6372
не только 0.5968
мы 0.5765
безопасности 0.5508
несмотря 0.538
несмотря на 0.538
включая 0.5303
остается 0.5263
эксперты 0.5118
только 0.5086
подобные 0.5075
особенно 0.4755
внимание 0.4708
произошедшего 0.4668
данный 0.4623
продолжает 0.4407

🔹 Маркеры Human:
сообщает -0.7498
об -0.6957
об этом -0.6913
этом -0.661
процентов -0.6268
процента -0.5356
года -0.4561
со ссылкой -0.4047
ссылкой на -0.4041
ссылкой -0.4035
риа новости -0.4011
риа -0.4011
при этом -0.3787
новости -0.3766
the -0.3752
пишет -0.37
этом сообщает -0.37
сообщалось -0.3636
сказал -0.3621
президента -0.3596


In [8]:
stylo_importance = model.named_steps['clf'].coef_[0][-4:]
stylo_feats = ["avg_sent_len","sent_len_var","ttr","markdown_bold"]

print("Стилометрия (влияние на класс AI):")
for f, w in zip(stylo_feats, stylo_importance):
    print(f"{f}: {round(w, 4)}")


Стилометрия (влияние на класс AI):
avg_sent_len: 1.0644
sent_len_var: -1.7677
ttr: -0.8524
markdown_bold: 2.5615


# функция проверки модели

In [9]:
label_map = {1: "AI-generated", 0: "Human-written"}

def classify_text(text: str):
    sample = pd.DataFrame([{
        "text": text,
        **extract_stylo_features(text)
    }])
    pred = model.predict(sample)[0]
    proba = model.predict_proba(sample)[0][1]
    label = label_map[pred]
    confidence = proba if pred == 1 else 1 - proba
    return {"prediction": label, "confidence": float(confidence)}

# пример
example = '''Москва, 30 ноября 2025 года. Росатом объявил о вводе в промышленную эксплуатацию плавучей атомной теплоэлектростанции (ПАТЭС) проекта 2025 «Академик Ломоносов-2» в акватории Певека (Чукотский АО).
Новая станция оснащена двумя реакторами последнего поколения РИТМ-200М мощностью по 60 МВт каждый и способна одновременно вырабатывать до 120 МВт электроэнергии и до 100 Гкал/ч тепла. По сравнению с первой ПАТЭС («Академик Ломоносов», запущен в 2019 году), новая станция на 30 % компактнее, на 20 % экономичнее по топливу и имеет увеличенный срок эксплуатации до 60 лет без перезарядки.'''
print(classify_text(example))


{'prediction': 'Human-written', 'confidence': 0.9510883089432539}


In [ ]:
import joblib
joblib.dump(model, "model.pkl")

['model.pkl']